In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
import analyze_covid
import airport_data_explore

# Unload stored dataframes from other notebooks within directory
%store -r

/home/aytang/sfuhome/353project/airport_data_explore.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  canada['Date'] = pd.to_datetime(canada['Date'])


In [2]:
# Haversine Caluclation (Distance between two lat and lon coordinates) retrieved from:
# https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula/21623206

def haversine(lat1, lon1, lat2, lon2):
    p = np.pi / 180
    a = 0.5 - np.cos((lat2 - lat1) * p) / 2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p )) / 2
    n = 12742 * np.arcsin(np.sqrt(a))
    return n  

In [3]:
def getStates():
    return usLocations['Province/State'].unique()
getStates()

array(['New York', 'Washington', 'New Jersey', 'California', 'Illinois',
       'Michigan', 'Florida', 'Louisiana', 'Massachusetts', 'Texas',
       'Georgia', 'Colorado', 'Pennsylvania', 'Tennessee', 'Wisconsin',
       'Ohio', 'Connecticut', 'North Carolina', 'Maryland', 'Virginia',
       'Mississippi', 'Indiana', 'South Carolina', 'Nevada', 'Utah',
       'Minnesota', 'Arkansas', 'Oregon', 'Alabama', 'Arizona',
       'Kentucky', 'Missouri', 'Iowa', 'Maine', 'Rhode Island',
       'New Hampshire', 'Oklahoma', 'Kansas', 'New Mexico', 'Hawaii',
       'Delaware', 'Vermont', 'Nebraska', 'Idaho', 'Montana',
       'North Dakota', 'Wyoming', 'South Dakota', 'Alaska',
       'West Virginia', 'District of Columbia'], dtype=object)

In [212]:
def getAirports(state):
    df = airportLocationsUS[airportLocationsUS['State'] == state]
    return df.to_dict('records')
getAirports('Hawaii')
t = usLocations[usLocations["Province/State"] == "California"]
i = airportLocationsUS[airportLocationsUS["State"] == "California"]
i

,AirportName,City,State,Country,lat,long
0,Los Angeles International,Los Angeles,California,United States of America (the),33.941369,-118.404993
1,San Francisco International,South San Francisco,California,United States of America (the),37.621188,-122.383935


In [218]:
def closest(state):
    d = []
    
    t = usLocations[usLocations["Province/State"] == state]
    i = airportLocationsUS[airportLocationsUS["State"] == state]
    size = len(t)
    size2 = len(i)

    for a in range(size2):
        for s in range(size):
            airport = {
                    'country': t.iloc[s]["Country/Region"],
                    'state/province': t.iloc[s]["Province/State"],
                    'county': t.iloc[s]["County"],
                    'lat': t.iloc[s]["Latitude"],
                    'long': t.iloc[s]["Longitude"],
                    'airport': i.iloc[a]["AirportName"],
                    'distance(km)': haversine(t.iloc[s]["Latitude"], t.iloc[s]["Longitude"], i.iloc[a]["lat"], i.iloc[a]["long"])
                }
            d.append(airport)
    d = sorted(d, key = lambda x: x['distance(km)'])
    return d

In [217]:
def calcNearestAirport(state):
    d = []
    for s in state:
        d.append(closest(state))
    return d[0]

calcNearestAirport('California')        

[{'country': 'US',
  'state/province': 'California',
  'county': 'San Francisco',
  'lat': 37.75215114,
  'long': -122.4385672,
  'airport': 'San Francisco International',
  'distance (km)': 15.335485129595366},
 {'country': 'US',
  'state/province': 'California',
  'county': 'San Francisco',
  'lat': 37.75215114,
  'long': -122.43856720000001,
  'airport': 'San Francisco International',
  'distance (km)': 15.335485129595366},
 {'country': 'US',
  'state/province': 'California',
  'county': 'San Mateo',
  'lat': 37.4228808,
  'long': -122.32755459999998,
  'airport': 'San Francisco International',
  'distance (km)': 22.60435060036342},
 {'country': 'US',
  'state/province': 'California',
  'county': 'San Mateo',
  'lat': 37.422880799999994,
  'long': -122.32755459999998,
  'airport': 'San Francisco International',
  'distance (km)': 22.60435060036342},
 {'country': 'US',
  'state/province': 'California',
  'county': 'Alameda',
  'lat': 37.64629437,
  'long': -121.8929271,
  'airport': 

In [242]:
statesList = getStates()
listDf = []

for s in statesList:
    i = calcNearestAirport(s)
    listDf.append(pd.DataFrame.from_dict(i))

main = pd.concat(listDf)

In [243]:
main 

,airport,country,county,distance(km),lat,long,state/province
0,John F. Kennedy International,US,Queens,7.905192,40.710881,-73.816847,New York
1,LaGuardia,US,New York City,8.308939,40.767273,-73.971526,New York
2,LaGuardia,US,New York,8.308939,40.767273,-73.971526,New York
3,LaGuardia,US,Queens,8.464824,40.710881,-73.816847,New York
4,LaGuardia,US,Bronx,8.740585,40.852093,-73.862828,New York
5,John F. Kennedy International,US,Kings,14.461959,40.636182,-73.949356,New York
6,John F. Kennedy International,US,Kings,14.461959,40.636183,-73.949356,New York
7,LaGuardia,US,Kings,16.601418,40.636182,-73.949356,New York
8,LaGuardia,US,Kings,16.601418,40.636183,-73.949356,New York
9,John F. Kennedy International,US,Nassau,19.097665,40.740665,-73.589419,New York


In [238]:
# TODO: Cluster counties into their respective/common airport.
# If > 1 airport in a state, group the county to the nearest airport.
# Group all counties in the same state to the same county as well?